#**DATA ACUISITION and MANIPULATION**

In [1]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


Extracting only US States data from the world Influenza-like-Illness(ILI) dataset and merging it with December 2020 pollutants data

In [18]:
import pandas as pd
import numpy as np
from datetime import timedelta

# 1. Load Data
ilin_df = pd.read_csv("/content/drive/MyDrive/ILINet.csv", header=1)
aq_df = pd.read_csv("/content/drive/MyDrive/forecasts_Dec_2020_df.csv")

# 2. Prepare and Expand ILINet Data to Daily Rows (Target + Temporal Features)
# Converting ~8,500 weekly records to ~60,000 daily records.
ilin_df = ilin_df[ilin_df['REGION TYPE'] == 'States'].copy()
ilin_df.rename(columns={'REGION': 'state_name'}, inplace=True)

# Clean up ILI target variable
ilin_df.replace('X', np.nan, inplace=True)
ilin_df['%UNWEIGHTED ILI'] = pd.to_numeric(ilin_df['%UNWEIGHTED ILI'], errors='coerce')
ilin_df.dropna(subset=['%UNWEIGHTED ILI'], inplace=True)
ilin_df['state_name'] = ilin_df['state_name'].str.strip()

# Create a weekly date for joining (Week Ending Date - Saturday)
# Using pandas ISO calendar to create the date for the week *starting* on Monday of that week.
def get_week_ending_date(row):
    try:
        # Create date object for Thursday of the given ISO year/week (mid-week)
        # Using 4 for Thursday, as it is always in the specified week number
        # Adjust to Saturday (Week Ending Date: +2 days)
        return pd.to_datetime(f"{int(row['YEAR'])}-{int(row['WEEK'])}-4", format="%Y-%W-%w") + timedelta(days=2)
    except ValueError:
        return np.nan

# Handle edge cases (Week 53) manually to ensure the correct year assignment
ilin_df.loc[((ilin_df['WEEK'] == 53) & (ilin_df['YEAR'] == 2010)), 'WEEK'] = 52
ilin_df.loc[((ilin_df['WEEK'] == 53) & (ilin_df['YEAR'] == 2021)), 'WEEK'] = 52
ilin_df['WEEKEND'] = ilin_df.apply(get_week_ending_date, axis=1)

# Backfill NaN dates for robustness by using the standard CDC convention:
# Week 1 starts on the first Sunday of the year, and the week ends on the following Saturday.
# A simpler method for creating the week ending date, which avoids the year/week/day ISO issues:
# 1. Find the first day of the year/week.
# 2. Add 6 days to get the Saturday end-date.
ilin_df.dropna(subset=['WEEKEND'], inplace=True)
ilin_df.rename(columns={'WEEKEND': 'Week Ending Date'}, inplace=True)

# Convert weekly records to daily records (replication)
daily_records = []
for index, row in ilin_df.iterrows():
    end_date = row['Week Ending Date']
    start_date = end_date - timedelta(days=6)

    # Replicate the weekly data for 7 days
    dates = pd.date_range(start=start_date, end=end_date, freq='D')

    for date in dates:
        daily_records.append({
            'state_name': row['state_name'],
            'date': date,
            'Year_ili': row['YEAR'],
            'Week_ili': row['WEEK'],
            'Week Ending Date': end_date,
            'ILI_Target': row['%UNWEIGHTED ILI'],
            'Total_Patients': row['TOTAL PATIENTS'],
            'Num_Providers': row['NUM. OF PROVIDERS']
        })

daily_ili_df = pd.DataFrame(daily_records)
daily_ili_df['state_name'] = daily_ili_df['state_name'].str.strip()


# --- 3. Prepare AQ Daily Data (Features) ---
# Aggregate county-level AQ data to state-level averages to match the ILINet data
aq_df['date'] = pd.to_datetime(aq_df['date'])
aq_df['state_name'] = aq_df['state_name'].str.strip()

aq_cols = ['co_polutant_level', 'no2_polutant_level', 'o3_polutant_level',
           'pm10_polutant_level', 'pm25frm_polutant_level', 'pm25nfrm_polutant_level',
           'so2_polutant_level']
for col in aq_cols:
    aq_df[col] = pd.to_numeric(aq_df[col], errors='coerce')

# Aggregate to state-level daily means
aq_daily_state_avg = aq_df.groupby(['state_name', 'date'])[aq_cols].mean().reset_index()

# --- 4. Merge Expanded ILINet Data (Target) with AQ Data (Features) ---
# Left merge: Daily ILINet data is the core structure. AQ features are added where available (Dec 2020).
final_merged_df = pd.merge(daily_ili_df, aq_daily_state_avg, on=['state_name', 'date'], how='left')

# --- 5. Final Cleanup and Output ---
final_merged_df['Day_of_Week_Name'] = final_merged_df['date'].dt.day_name()

# Select final columns
final_cols = [
    'state_name', 'date', 'Day_of_Week_Name', 'Week Ending Date', 'Year_ili', 'Week_ili',
    'ILI_Target',
    'Total_Patients', 'Num_Providers',
    'co_polutant_level', 'no2_polutant_level', 'o3_polutant_level', 'pm10_polutant_level',
    'pm25frm_polutant_level', 'pm25nfrm_polutant_level', 'so2_polutant_level',
]

final_merged_df = final_merged_df[final_cols]

# Save the final file
output_file_name = "full_daily_sparse_features_ili_target_US_states.csv"
final_merged_df.to_csv(output_file_name, index=False)
print(f"File created: {output_file_name}")
print(f"Final Merged Dataset size (rows): {len(final_merged_df)}")

/tmp/ipython-input-1891356651.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ilin_df.replace('X', np.nan, inplace=True)


File created: full_daily_sparse_features_ili_target_US_states.csv
Final Merged Dataset size (rows): 295659


Since the ILI data was being too large and complex and the pollutant data was sparse for continuous years, we decided to filter data from 2015 to 2025

In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta

# 1. Load the file
try:
    df = pd.read_csv("/content/drive/MyDrive/full_daily_sparse_features_ili_target_US_states.csv")
except FileNotFoundError:
    print("Error: The file full_daily_sparse_features_ili_target_US_states.csv was not found.")
    df = pd.DataFrame() # Create an empty DataFrame to avoid further errors

# 2. Ensure 'date' is datetime and filter
if not df.empty:
    df['date'] = pd.to_datetime(df['date'])

    # 3. Filter data for 2015 to 2025 (inclusive)
    # The upper limit is 2025, which is the latest data point available.
    df_filtered = df[df['date'].dt.year >= 2015].copy()

    # 4. Save the filtered DataFrame
    output_file_name = "full_daily_sparse_features_ili_target_US_states_2015_2025.csv"
    df_filtered.to_csv(output_file_name, index=False)

    # Report size
    print(f"Filtered Dataset size (rows): {len(df_filtered)}")
    print(f"File created: {output_file_name}")
else:
    print("Cannot process the request because the source file could not be loaded.")

Filtered Dataset size (rows): 213739
File created: full_daily_sparse_features_ili_target_US_states_2015_2025.csv


The ILI data was completely manipulated but the AQ data was for December 2020 and Weather data was entirely missing. So after finding the merged AQ and weather data from the year 2015 to 2025 of US states, we merged it to the ILI data based on state and year

In [8]:
import pandas as pd
import numpy as np
import os

# 1. Load Main Daily File (2015–2025)
main_df = pd.read_csv("/content/drive/MyDrive/full_daily_features_with_all_pollutants_2015_2025.csv")

main_df['date'] = pd.to_datetime(main_df['date'], errors='coerce')
main_df['state_name'] = main_df['state_name'].str.strip()

# Helper Function for Pollutant ID
def get_pollutant_id(name):
    if 'PM2.5' in name:
        return 'PM2.5'
    elif 'PM10' in name:
        return 'PM10'
    elif 'Light Absorption Coeffiecient' in name:
        return 'Light_Absorption_Coeffiecient'
    elif 'Average Ambient Temperature' in name:
        return 'Average_Ambient_Temperature'
    elif 'Average Ambient Pressure' in name:
        return 'Average_Ambient_Pressure'
    return 'Other_Max_Annual_Reading'

# 2. Loop Through Multiple Years of Monitor Files
all_years_df = []   # To store processed data for each year

for year in range(2015, 2026):   # 2015 to 2025
    file_path = f"/content/drive/MyDrive/annual_conc_by_monitor_{year}.csv"
    if not os.path.exists(file_path):
        print(f"❌ File not found for {year}: {file_path}")
        continue
    print(f"✅ Processing: {file_path}")

    # Load the file
    monitor_df = pd.read_csv(file_path)

    # Rename columns
    monitor_df.rename(columns={
        'State Name': 'state_name',
        '1st Max Value': 'Max_Value',
        '1st Max DateTime': 'Max_DateTime'
    }, inplace=True)

    # Filter to keep only PM-related parameters
    pm_parameters = monitor_df[
        monitor_df['Parameter Name'].str.contains(
            'PM2.5|PM10|Light Absorption Coeffiecient|Average Ambient Pressure|Average Ambient Temperature',
            na=False
        ) &
        ~monitor_df['Parameter Name'].str.contains('NO2|SO2|CO|Ozone', na=False)
    ].copy()

    pm_parameters['state_name'] = pm_parameters['state_name'].str.strip()
    pm_parameters['date'] = pd.to_datetime(pm_parameters['Max_DateTime'], errors='coerce').dt.date

    # Pollutant ID column
    pm_parameters['Pollutant_ID'] = pm_parameters['Parameter Name'].apply(get_pollutant_id)

    # Group and compute max for each pollutant in each state & date
    max_pm_pivot = pm_parameters.groupby(
        ['state_name', 'date', 'Pollutant_ID']
    )['Max_Value'].max().reset_index()

    # Pivot to wide format
    max_pm_final = max_pm_pivot.pivot_table(
        index=['state_name', 'date'],
        columns='Pollutant_ID',
        values='Max_Value',
        aggfunc='max'
    ).reset_index()

    max_pm_final['date'] = pd.to_datetime(max_pm_final['date'])

    all_years_df.append(max_pm_final)

# 3. Combine All Years
if all_years_df:
    combined_max_pm = pd.concat(all_years_df, ignore_index=True)
else:
    raise ValueError("No annual monitor files were successfully processed.")

print("🔄 Combined annual monitor shape:", combined_max_pm.shape)

# 4. Merge With Main DF
final_merged_df = pd.merge(
    main_df,
    combined_max_pm,
    on=['state_name', 'date'],
    how='left'
)

# 5. Save Final File
output_file = "/content/drive/MyDrive/full_daily_features_with_PM_max_2015_2025_FINAL.csv"
final_merged_df.to_csv(output_file, index=False)

print("🎉 File created:", output_file)
print("📌 Final rows:", len(final_merged_df))

✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2015.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2016.csv


/tmp/ipython-input-808240213.py:45: DtypeWarning: Columns (38,40,53) have mixed types. Specify dtype option on import or set low_memory=False.
  monitor_df = pd.read_csv(file_path)


✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2017.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2018.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2019.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2020.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2021.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2022.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2023.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2024.csv
✅ Processing: /content/drive/MyDrive/annual_conc_by_monitor_2025.csv
🔄 Combined annual monitor shape: (49523, 7)
🎉 File created: /content/drive/MyDrive/full_daily_features_with_PM_max_2015_2025_FINAL.csv
📌 Final rows: 213739


Visualizing the final merged dataset

In [15]:
df = pd.read_csv("/content/drive/MyDrive/daily_AQ_2015_2025.csv")
daily_cols = [
    'PM2.5_pollutant_level', 'PM10_pollutant_level',
    'carbon_monoxide_pollutant_level', 'ozone_pollutant_level',
    'nitrogen_dioxide_pollutant_level', 'sulfur_dioxide_pollutant_level'
]

         state_name        date Day_of_Week_Name Week Ending Date  Year_ili  \
0           Alabama  01-01-2015         Thursday       03-01-2015      2014   
1           Alabama  02-01-2015           Friday       03-01-2015      2014   
2           Alabama  03-01-2015         Saturday       03-01-2015      2014   
3            Alaska  01-01-2015         Thursday       03-01-2015      2014   
4            Alaska  02-01-2015           Friday       03-01-2015      2014   
...             ...         ...              ...              ...       ...   
213734  Puerto Rico  18-11-2025          Tuesday       22-11-2025      2025   
213735  Puerto Rico  19-11-2025        Wednesday       22-11-2025      2025   
213736  Puerto Rico  20-11-2025         Thursday       22-11-2025      2025   
213737  Puerto Rico  21-11-2025           Friday       22-11-2025      2025   
213738  Puerto Rico  22-11-2025         Saturday       22-11-2025      2025   

        Week_ili  ILI_Target  Total_Patients  Num_P

Handling Missing Values

In [25]:
import pandas as pd
import numpy as np

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/daily_AQ_2015_2025.csv')

# Convert date columns to datetime for sorting and potential time-based operations
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y', errors='coerce')
df['Week Ending Date'] = pd.to_datetime(df['Week Ending Date'], format='%d-%m-%Y', errors='coerce')

# Sort by state and date to enable proper forward-fill
df = df.sort_values(['state_name', 'date'])

# Identify categorical columns for forward-fill
categorical_cols = ['state_name', 'Day_of_Week_Name', 'Week Ending Date', 'Year_ili', 'Week_ili']

# Identify daily pollutant columns for state-wise median imputation
daily_pollutant_cols = [
    'ozone_pollutant_level', 'sulfur_dioxide_pollutant_level', 'carbon_monoxide_pollutant_level',
    'nitrogen_dioxide_pollutant_level', 'Average_Ambient_Pressure', 'Average_Ambient_Temperature',
    'PM10_pollutant_level', 'PM2.5_pollutant_level', 'Light_Absorption_Coeffiecient'
]

# Other numerical columns (weekly health metrics, repeated daily; forward-fill as alternative if needed, but median here for consistency)
other_numerical_cols = ['ILI_Target', 'Total_Patients', 'Num_Providers']

# Add missing-value indicator flags for all imputable numerical columns
imputable_cols = daily_pollutant_cols + other_numerical_cols
for col in imputable_cols:
    if df[col].isnull().any():
        df[f'{col}_was_missing'] = df[col].isnull().astype(int)

# Impute categorical columns with forward-fill within each state
for col in categorical_cols:
    df[col] = df.groupby('state_name')[col].ffill()

# Impute daily pollutant columns with state-wise median
for col in daily_pollutant_cols:
    state_medians = df.groupby('state_name')[col].median()
    df[col] = df.apply(
        lambda row: state_medians.get(row['state_name'], np.nan) if pd.isnull(row[col]) else row[col],
        axis=1
    )

# For other numerical (weekly) cols, use state-wise median as well (since they may vary less)
for col in other_numerical_cols:
    state_medians = df.groupby('state_name')[col].median()
    df[col] = df.apply(
        lambda row: state_medians.get(row['state_name'], np.nan) if pd.isnull(row[col]) else row[col],
        axis=1
    )

# If any NaNs remain (e.g., entire state missing for a col), fall back to global median
for col in imputable_cols:
    if df[col].isnull().any():
        global_median = df[col].median()
        df[col] = df[col].fillna(global_median)

# Verify no NaNs left
print("Missing values after imputation:\n", df.isnull().sum())

# Optionally save the imputed dataset
df.to_csv('imputed_daily_AQ_2015_2025.csv', index=False)

# Preview the imputed DataFrame
print(df.head())

Missing values after imputation:
 state_name                                      0
date                                            0
Day_of_Week_Name                                0
Week Ending Date                                0
Year_ili                                        0
Week_ili                                        0
ILI_Target                                      0
Total_Patients                                  0
Num_Providers                                   0
ozone_pollutant_level                           0
sulfur_dioxide_pollutant_level                  0
carbon_monoxide_pollutant_level                 0
nitrogen_dioxide_pollutant_level                0
Average_Ambient_Pressure                        0
Average_Ambient_Temperature                     0
PM10_pollutant_level                            0
PM2.5_pollutant_level                           0
Light_Absorption_Coeffiecient                   0
ozone_pollutant_level_was_missing               0
sulfur_dioxide_p

#REASON:

Before imputation, we create missing-value indicator flags for all numerical columns that can be imputed.
This allows us to track which values were originally missing, which can be useful for later analysis or modeling.

##State-wise Median

Daily pollutant and environmental numerical columns are imputed using the median for each state.
Median is used because it is robust to outliers, which are common in environmental data.
This ensures that the imputed values reflect typical conditions for that state rather than being skewed by extremes.

##Global Median

As a final safeguard, if any NaNs remain (for example, if an entire state had missing data for a column),
we fill those remaining missing values with the global median of that column.
This ensures that there are no missing values left in the dataset.